# Pre-process_Xenium_Prime_Human_Lymph_Node_Reactive_FFPE_outs

In [1]:
%load_ext autoreload
%autoreload 2
%env ANYWIDGET_HMR=1

env: ANYWIDGET_HMR=1


In [2]:
import numpy as np
import pandas as pd
import celldega as dega
import tifffile
import zarr
import matplotlib.pyplot as plt
from matplotlib.colors import to_hex
import geopandas as gpd
import shapely
import tarfile

# macOS requirement
import os
os.environ['DYLD_LIBRARY_PATH'] = '/opt/homebrew/lib:' + os.environ.get('DYLD_LIBRARY_PATH', '')

In [ ]:
# pass values to the following variables

dataset_name = 'Xenium_Prime_Human_Lymph_Node_Reactive_FFPE_outs'
base_path = 'data/xenium_data/' + dataset_name + '/'
path_landscape_files = 'data/xenium_landscapes/' + dataset_name + '_landscape_files/'

# zipped files are not always present in xenium
# input variable of celldega version

tile_size = 250
image_scale = 0.5
technology = 'Xenium'

suffix = '.webp[Q=100]'

if not os.path.exists(path_landscape_files):
    os.mkdir(path_landscape_files)

if not os.path.exists(path_landscape_files + 'cell_clusters/'):
    os.mkdir(path_landscape_files + 'cell_clusters/')

## Unzip Xenium Data

#### Decompress Cell Feature Matrix MTX Files

In [11]:
# Path to the tar.gz file you want to decompress
tar_file_path = base_path + 'cell_feature_matrix.tar.gz'

# Open the tar.gz file
with tarfile.open(tar_file_path, "r:gz") as tar:
    # Extract all contents to the specified directory
    tar.extractall(path=path_landscape_files)

print(f"File {tar_file_path} has been decompressed to {path_landscape_files}")

File ../data/xenium_data/Xenium_Prime_Human_Lymph_Node_Reactive_FFPE_outs/cell_feature_matrix.tar.gz has been decompressed to ../data/xenium_landscapes/Xenium_Prime_Human_Lymph_Node_Reactive_FFPE_outs_landscape_files/


#### Decompress Xenium Analysis Files

In [12]:
# Path to the tar.gz file you want to decompress
tar_file_path = base_path + 'analysis.tar.gz'

# Open the tar.gz file
with tarfile.open(tar_file_path, "r:gz") as tar:
    # Extract all contents to the specified directory
    tar.extractall(path=path_landscape_files)

print(f"File {tar_file_path} has been decompressed to {path_landscape_files}")

File ../data/xenium_data/Xenium_Prime_Human_Lymph_Node_Reactive_FFPE_outs/analysis.tar.gz has been decompressed to ../data/xenium_landscapes/Xenium_Prime_Human_Lymph_Node_Reactive_FFPE_outs_landscape_files/


# CBG

In [13]:
cbg = dega.pre.read_cbg_mtx(path_landscape_files + 'cell_feature_matrix/')

read mtx file from  ../data/xenium_landscapes/Xenium_Prime_Human_Lymph_Node_Reactive_FFPE_outs_landscape_files/cell_feature_matrix/


1,A2ML1,AAMP,AAR2,AARSD1,ABAT,ABCA1,ABCA3,ABCA4,ABCB1,ABCB4,...,DeprecatedCodeword_18589,DeprecatedCodeword_18601,DeprecatedCodeword_18609,DeprecatedCodeword_18616,DeprecatedCodeword_18620,DeprecatedCodeword_18632,DeprecatedCodeword_18637,DeprecatedCodeword_18639,DeprecatedCodeword_18646,DeprecatedCodeword_18649
0,,,,,,,,,,,,,,,,,,,,,
aaaaadoa-1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aaaaclhf-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aaaafcfj-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aaaagamp-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aaaaiako-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
oikoinno-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
oikojopg-1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
oikokaeg-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
meta_gene_exp = dega.pre.calc_meta_gene_data(cbg)

calculating mean expression from sparse float data
calculating variance by looping over rows


### Gene Metadata

In [15]:
path_cbg = path_landscape_files + 'cell_feature_matrix/'
path_output = path_landscape_files + 'meta_gene.parquet'
dega.pre.make_meta_gene(technology, path_cbg, path_output)

read mtx file from  ../data/xenium_landscapes/Xenium_Prime_Human_Lymph_Node_Reactive_FFPE_outs_landscape_files/cell_feature_matrix/
calculating mean expression from sparse float data
calculating variance by looping over rows


### Cell-by-gene Files

In [17]:
dega.pre.save_cbg_gene_parquets(path_landscape_files, cbg, verbose=True)

### Image Tiles

In [ ]:
# Path to your OME-TIFF file
file_path = base_path + 'morphology_focus/morphology_focus_0000.ome.tif'

# Open the OME-TIFF file and read the image data
with tifffile.TiffFile(file_path) as tif:
    series = tif.series[0]  # Assuming you are interested in the first series
    image_data = series.asarray()

### DAPI

In [34]:
for channel_index, channel_name in zip(range(image_data.shape[0]), ['dapi','bound','rna','prot']):

    image_data_scaled = image_data[channel_index,:,:] * 2
    
    # Save the image data to a regular TIFF file without compression
    tifffile.imwrite(path_landscape_files + 'output_regular.tif', image_data_scaled, compression=None)
    image_ds = dega.pre.reduce_image_size(path_landscape_files + 'output_regular.tif', image_scale, path_landscape_files)
    image_png = dega.pre.convert_to_png(image_ds)
    dega.pre.make_deepzoom_pyramid(image_png, path_landscape_files + 'pyramid_images/', channel_name, suffix=suffix)

### Cell Metadata

In [26]:
# Function to open a Zarr file
def open_zarr(path: str) -> zarr.Group:
    store = (zarr.ZipStore(path, mode="r")
    if path.endswith(".zip")
    else zarr.DirectoryStore(path)
    )
    return zarr.group(store=store)

# For example, use the above function to open the cells Zarr file, which contains segmentation mask Zarr arrays
root = open_zarr(base_path + "cells.zarr.zip")
transformation_matrix = root['masks']['homogeneous_transform'][:]

In [28]:
pd.DataFrame(transformation_matrix[:3,:3]).to_csv(
    path_landscape_files + 'xenium_transform.csv', 
    sep=' ', 
    header=False, 
    index=False
)

In [11]:
path_transformation_matrix = path_landscape_files + 'xenium_transform.csv'
path_meta_cell_micron = base_path + 'cells.csv.gz'
path_meta_cell_image = path_landscape_files + 'cell_metadata.parquet'

In [12]:
default_clustering = pd.read_csv(path_landscape_files + 'analysis/clustering/gene_expression_graphclust/clusters.csv', index_col=0)

,Cluster
Barcode,
aaaaadoa-1,19
aaaaclhf-1,6
aaaafcfj-1,2
aaaagamp-1,19
aaaaiako-1,10
...,...
oikoinno-1,7
oikojopg-1,3
oikokaeg-1,13


### Save default clustering results

In [29]:
default_clustering_ini = pd.DataFrame(default_clustering.values, index=default_clustering.index.tolist(), columns=['cluster'])
default_clustering_ini['cluster'] = default_clustering_ini['cluster'].astype('string')
default_clustering = pd.DataFrame(index=meta_cell.index.tolist())
default_clustering.loc[default_clustering_ini.index.tolist(), 'cluster'] = default_clustering_ini['cluster']
default_clustering.to_parquet(path_landscape_files + 'cell_clusters/cluster.parquet')

ser_counts = default_clustering['cluster'].value_counts()
clusters = ser_counts.index.tolist()

,cluster
aaaaadoa-1,19
aaaaclhf-1,6
aaaafcfj-1,2
aaaagamp-1,19
aaaaiako-1,10


### Save cell metadata

In [17]:
# do not including clustering information in default cell metadata
dega.pre.make_meta_cell_image_coord(
    technology, 
    path_transformation_matrix, 
    path_meta_cell_micron, 
    path_meta_cell_image, 
)

### Cluster Colors

In [19]:
# Get all categorical color palettes from Matplotlib and flatten them into a single list of colors
palettes = [plt.get_cmap(name).colors for name in plt.colormaps() if "tab" in name]
flat_colors = [color for palette in palettes for color in palette]

# Convert RGB tuples to hex codes
flat_colors_hex = [to_hex(color) for color in flat_colors]

# Use modular arithmetic to assign a color to each gene, white for genes with "Blank"
colors = [
    flat_colors_hex[i % len(flat_colors_hex)] if "Blank" not in cluster else "#FFFFFF"
    for i, cluster in enumerate(clusters)
]

# Create a DataFrame with genes and their assigned colors
ser_color = pd.Series(colors, index=clusters, name='color')

meta_cluster = pd.DataFrame(ser_color)

meta_cluster['count'] = ser_counts

meta_cluster.to_parquet(path_landscape_files + 'cell_clusters/meta_cluster.parquet')

### Transcripts

In [ ]:
%%time 

path_trx = base_path + 'transcripts.parquet'
path_trx_tiles = path_landscape_files + 'transcript_tiles'
tile_bounds = dega.pre.make_trx_tiles(
    technology, 
    path_trx, 
    path_transformation_matrix, 
    path_trx_tiles,
    tile_size=tile_size,
    image_scale=image_scale
)

### Cell Boundaries

In [35]:
%%time
path_cell_boundaries = base_path + 'cell_boundaries.parquet'
path_output = path_landscape_files + 'cell_segmentation'
dega.pre.make_cell_boundary_tiles(
    technology,
    path_cell_boundaries, 
    path_meta_cell_micron, 
    path_transformation_matrix, 
    path_output,
    tile_size=tile_size,
    tile_bounds=tile_bounds
)

CPU times: user 11min 24s, sys: 1min 4s, total: 12min 28s
Wall time: 12min 41s


### Gene Metadata

In [36]:
path_cbg = path_landscape_files + 'cell_feature_matrix/'
path_output = path_landscape_files + 'gene_metadata.parquet'
dega.pre.make_meta_gene(technology, path_cbg, path_output)

read mtx file from  ../data/xenium_landscapes/Xenium_Prime_Human_Lymph_Node_Reactive_FFPE_outs_landscape_files/cell_feature_matrix/
calculating mean expression from sparse float data
calculating variance by looping over rows


### Cluster Gene Expression

In [39]:
usecols = ['cell_id', 'x_centroid', 'y_centroid']
meta_cell = pd.read_csv(base_path + 'cells.csv.gz', index_col=0, usecols=usecols)
meta_cell.columns = ['center_x', 'center_y']

df_meta = pd.read_csv(path_landscape_files + 'analysis/clustering/gene_expression_graphclust/clusters.csv', index_col=0)
df_meta['Cluster'] = df_meta['Cluster'].astype('string')
df_meta.columns = ['cluster']

meta_cell['cluster'] = df_meta['cluster']

,center_x,center_y
cell_id,,
aaaaadoa-1,2871.859619,347.729767
aaaaclhf-1,2882.301025,349.938110
aaaafcfj-1,2880.217041,338.575897
aaaagamp-1,2852.795166,356.880615
aaaaiako-1,2854.036133,361.754639
...,...,...
oikoinno-1,6281.145996,4459.715332
oikojopg-1,6231.592285,4434.088379
oikokaeg-1,6243.616211,4449.573730


In [41]:
list_ser = []
for inst_cat in meta_cell['cluster'].unique().tolist():
    if inst_cat is not None:
        inst_cells = meta_cell[meta_cell['cluster'] == inst_cat].index.tolist()

        inst_ser = cbg.loc[inst_cells].sum()/len(inst_cells)
        inst_ser.name = inst_cat

        list_ser.append(inst_ser)

df_sig = pd.concat(list_ser, axis=1)    

In [42]:
# handling weird behavior where there is a multiindex it appears

df_sig.columns = df_sig.columns.tolist()
df_sig.index = df_sig.index.tolist()

In [47]:
keep_genes = df_sig.index.tolist()
keep_genes = [x for x in keep_genes if 'Unassigned' not in x]
keep_genes = [x for x in keep_genes if 'NegControl' not in x]
keep_genes = [x for x in keep_genes if 'DeprecatedCodeword' not in x]

df_sig = df_sig.loc[keep_genes, clusters]

(4645, 35)

In [48]:
df_sig.sparse.to_dense().to_parquet(path_landscape_files + 'df_sig.parquet')

### Save Landscape Parameters JSON

In [30]:
image_info =  [
        {
            "name": "dapi",
            "button_name": "DAPI",
            "color": [
                0,
                0,
                255
            ]
        },
        {
            "name": "bound",
            "button_name": "BOUND",
            "color": [
                0,
                255,
                0
            ]
        },
        {
            "name": "rna",
            "button_name": "RNA",
            "color": [
                255,
                0,
                0
            ]
        },
        {
            "name": "prot",
            "button_name": "PROT",
            "color": [
                255,
                255,
                255
            ]
        }
    ]

In [39]:
dega.pre.save_landscape_parameters(
    technology, 
    path_landscape_files,
    'dapi_files',
    tile_size=100,
    image_info=image_info,
    image_format='.webp'
)

../data/xenium_landscapes/Xenium_Prime_Human_Lymph_Node_Reactive_FFPE_outs_landscape_files/pyramid_images/dapi_files/
